In [ ]:
import json

import requests

import my_config

class RestAPI:
    def __init__(self, base_url, endpoints):
        self.base_url = base_url
        self.endpoints = endpoints
        self.data = {}

    def fetch_data(self):
        for key, endpoint in self.endpoints.items():
            url = self.base_url + endpoint
            response = requests.get(url)

            if response.status_code == 200:
                self.data[key] = response.json()
            else:
                print(f"Error: {response.status_code}. Couldn't retrieve data for {key}")

US Bureau of Economic Analysis API

In [ ]:
all_bea_tables = {
    "T10503": "Real Gross Domestic Product, Expanded Detail, Quantity Indexes (A) (Q)",
    "T10105": "Gross Domestic Product (A) (Q)",
    "T10505": "Gross Domestic Product, Expanded Detail (A) (Q)",
    "T80103": "Real Gross Domestic Product, Quantity Indexes, Not Seasonally Adjusted (Q)",
    "T80104": "Price Indexes for Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80105": "Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80106": "Real Gross Domestic Product, Chained Dollars, Not Seasonally Adjusted (Q)",
    "T80200": "Gross Domestic Income by Type of Income, Not Seasonally Adjusted (Q)",
    "T11000": "Gross Domestic Income by Type of Income (A) (Q)",
    "T11000": "Gross Domestic Income by Type of Income (A) (Q)",

    "T20301": "Percent Change From Preceding Period in Real Personal Consumption Expenditures by Major Type of Product (A) (Q)",

    "T20100": "Personal Income and Its Disposition (A) (Q)",
    "T20600": "Personal Income and Its Disposition, Monthly (M)",
    "T20200A": "Wages and Salaries by Industry (A) (Q)",
    "T20200B": "Wages and Salaries by Industry (A) (Q)",
    "T20700A": "Wages and Salaries by Industry, Monthly (M)",
    "T20700B": "Wages and Salaries by Industry, Monthly (M)",

    "T40201": "Percent Change From Preceding Period in Real Exports and in Real Imports of Goods and Services by Type of Product (A) (Q)",
    "T40203": "Real Exports and Imports of Goods and Services by Type of Product, Quantity Indexes (A) (Q)",
    "T40204": "Price Indexes for Exports and Imports of Goods and Services by Type of Product (A) (Q)",
    "T40205": "Exports and Imports of Goods and Services by Type of Product (A) (Q)",

    "T50100": "Saving and Investment by Sector (A) (Q)",
    "T50203": "Real Gross and Net Domestic Investment by Major Type, Quantity Indexes (A)",
    "T50205": "Gross and Net Domestic Investment by Major Type (A)",
    "T50301": "Percent Change From Preceding Period in Real Private Fixed Investment by Type (A) (Q)",

    "T50705A": "Change in Private Inventories by Industry (A) (Q)",
    "T50705B": "Change in Private Inventories by Industry (A) (Q)",

    "T61600A": "Corporate Profits by Industry (A)",
    "T61600B": "Corporate Profits by Industry (A) (Q)",
    "T61600C": "Corporate Profits by Industry (A) (Q)",
    "T61600D": "Corporate Profits by Industry (A) (Q)",

    "T70201A": "Percent Change From Preceding Period in Real Auto Output (A) (Q)",
    "T70201B": "Percent Change From Preceding Period in Real Motor Vehicle Output (A) (Q)",
    "T70203A": "Real Auto Output, Quantity Indexes (A) (Q)",
    "T70203B": "Real Motor Vehicle Output, Quantity Indexes (A) (Q)",

    "T80300": "Federal Government Current Receipts and Expenditures, Not Seasonally Adjusted (Q)"
}

gdp_percent_changes = {
    "T10101" : "Percent Change From Preceding Period in Real Gross Domestic Product (A) (Q)",
    "T10107": "Percent Change From Preceding Period in Prices for Gross Domestic Product (A) (Q)",
    "T10108": "Contributions to Percent Change in the Gross Domestic Product Price Index (A) (Q)",
    "T10501": "Percent Change From Preceding Period in Real Gross Domestic Product, Expanded Detail (A) (Q)",
    "T80111": "Real Gross Domestic Product: Percent Change From Quarter One Year Ago, Not Seasonally Adjusted (Q)"
}

In [ ]:
start_year = 2019
end_year = 2020
bea_years = range(start_year, end_year + 1)

bea_years = ','.join(str(year) for year in bea_years)
bea_frequency = 'A,Q,M'
bea_table = 'T10101'

'''
BEA Endpoint Descriptions:
"gdp" - Percent Change From Preceding Period in Real Gross Domestic Product (A) (Q).
'''

bea_base_url = "https://apps.bea.gov/api/data/"

bea_endpoints = {
    'gdp': f"?&UserID={my_config.BEA_KEY}&DataSetName=NIPA&method=GetData&TableName={bea_table}&Frequency={bea_frequency}&Year={bea_years}",
    # 'all_nipa_tables': f"?&UserID={my_config.BEA_KEY}&DataSetName=NIPA&method=GetParameterValues&ParameterName=TableName"
}

bea_api = RestAPI(bea_base_url, bea_endpoints)
bea_api.fetch_data()

print(json.dumps(bea_api.data['gdp']['BEAAPI']['Results']['Data'], indent=4))
# print(json.dumps(bea_api.data['all_nipa_tables'], indent=4))

In [ ]:
import pandas as pd

gdp_data = json.dumps(bea_api.data['gdp']['BEAAPI']['Results']['Data'], indent=4)
gdp_df = pd.read_json(gdp_data)
gdp_df2 = pd.DataFrame(gdp_df, columns=['TimePeriod', 'DataValue', 'LineDescription', 'METRIC_NAME']
    ).rename(columns={'TimePeriod': 'date', 'DataValue': 'gdp (%)', 'METRIC_NAME': 'metric', 'LineDescription': 'description'}
    ).set_index('date')

# display(gdp_df)
print(gdp_df2.to_string())

In [ ]:
print(gdp_df2['description'].unique())

In [ ]:
legit_gdp_df = gdp_df2.loc[gdp_df2['description'] == "Gross domestic product"]
legit_gdp_df = legit_gdp_df.drop(columns=['description'])

In [ ]:
print(legit_gdp_df.to_string())

In [ ]:
# Save json output to a .json file.
json_api_2 = json.dumps(bea_api.data['annual_gdp']['BEAAPI']['Results']['Data'], indent=4)
func = open("dict_2.json","w")
func.write(json_api_2)
func.close()

US Treasury Department API

In [ ]:
'''
Treasury Endpoint Descriptions:
"interest_rates" - Average interest rates for marketable and non-marketable securities.
"cash_balance" - This table represents the Treasury General Account balance. All figures are rounded to the nearest million.
"transactions" - This table represents deposits and withdrawals from the Treasury General Account. 
"us_debt" - Outstanding U.S. debt on a daily basis. Measured to the penny.
All figures are rounded to the nearest million.
'''

treasury_base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/"
treasury_endpoints = {
    'interest_rates': 'v2/accounting/od/avg_interest_rates',
    'cash_balance': 'v1/accounting/dts/dts_table_1',
    'transactions': 'v1/accounting/dts/dts_table_2',
    'us_debt': 'v2/accounting/od/debt_to_penny',
    'balance_sheet': 'v2/accounting/od/balance_sheets'
}

treasury_api = RestAPI(treasury_base_url, treasury_endpoints)
treasury_api.fetch_data()

print(json.dumps(treasury_api.data, indent=4))

US Bureau of Labor Statistics API - really slow

In [ ]:
labor_base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

labor_endpoints = {
    'unemployment_rate': 'LNS14000000',
    # 'cpi': 'CUUR0000SA0'
}

labor_dates = {
    'startyear': '2005',
    'endyear': '2020'
}

labor_api = RestAPI(labor_base_url, labor_endpoints)
labor_api.fetch_data()

print(json.dumps(labor_api.data['unemployment_rate'], indent=4))

In [ ]:
import json

import pandas as pd
import requests

import my_config

class RestAPI:
    def __init__(self, base_url, endpoints):
        self.base_url = base_url
        self.endpoints = endpoints
        self.data = {}

    def fetch_data(self):
        for key, endpoint in self.endpoints.items():
            url = self.base_url + endpoint
            response = requests.get(url)
            if response.status_code == 200: self.data[key] = response.json()

bea_years = range(start_year, end_year + 1)

bea_years = ','.join(str(year) for year in bea_years)
bea_frequency = 'Q'
bea_dataset = 'NIPA'
bea_table = 'T10101'

bea_base_url = f"https://apps.bea.gov/api/data/?&UserID={my_config.BEA_KEY}"
bea_endpoints = {
    'gdp': f"&method=GetData \
            &DataSetName={bea_dataset} \
            &TableName={bea_table} \
            &Frequency={bea_frequency} \
            &Year={bea_years}"
}

bea_api = RestAPI(bea_base_url, bea_endpoints)
bea_api.fetch_data()

gdp_data = json.dumps(bea_api.data['gdp']['BEAAPI']['Results']['Data'])
gdp_df = pd.read_json(gdp_data)
gdp_df = (
    pd.DataFrame(gdp_df, columns=['TimePeriod', 'DataValue', 'METRIC_NAME', 'LineDescription'])
        .rename(columns={'TimePeriod': 'date', 'DataValue': 'gdp (%)', 'METRIC_NAME': 'metric'})
        .loc[gdp_df['LineDescription'] == "Gross domestic product"]
        .drop(columns=['LineDescription'])
)
display(gdp_df)